In [1]:
!pip install llama-index llama-index-llms-huggingface llama-index-llms-huggingface-api llama-index-embeddings-huggingface   --quiet
!pip install bitsandbytes transformers evaluate accelerate hazm  clean-text[gpl]  --quiet
!pip install docx2txt  --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 66.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 892.6/892.6 kB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [ ]:
# Restart Session
import os
os.kill(os.getpid(), 9)

In [4]:
model_id = "NousResearch/Meta-Llama-3-8B-Instruct"

In [5]:
from transformers import AutoTokenizer
from llama_index.llms.huggingface import HuggingFaceLLM
import torch
from transformers import BitsAndBytesConfig
from llama_index.core.prompts import PromptTemplate
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import Settings\
import hazm
from cleantext import clean
import re
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [7]:
tokenizer = AutoTokenizer.from_pretrained(
    model_id
)

# quantize to save memory
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

llm = HuggingFaceLLM(
    model_name= model_id,
    tokenizer_name= model_id ,
    context_window=3900,
    max_new_tokens=256,
    model_kwargs={"quantization_config": quantization_config},
    generate_kwargs={"temperature": 0.7, "top_k": 50, "top_p": 0.95},
    stopping_ids=stopping_ids,
    device_map="auto",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

...

model-00006-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00007-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/816M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [15]:
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/LaBSE")

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.22k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer.json:   0%|          | 0.00/9.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

In [9]:
def cleaning(text):
    cleanr = re.compile('<.*?>')
    wierd_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"
        u"\U0001F300-\U0001F5FF" 
        u"\U0001F680-\U0001F6FF" 
        u"\U0001F1E0-\U0001F1FF"  
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u'\U00010000-\U0010ffff'
        u"\u200d"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\u3030"
        u"\ufe0f"
        u"\u2069"
        u"\u2066"
        # u"\u200c"
        u"\u2068"
        u"\u2067"
        "]+", flags=re.UNICODE)
    cleantext = re.sub(cleanr, '', clean(text.strip(),
        fix_unicode=True,
        to_ascii=False,
        lower=True,
        no_line_breaks=True,
        no_urls=True,
        no_emails=True,
        no_phone_numbers=True,
        no_numbers=False,
        no_digits=False,
        no_currency_symbols=True,
        no_punct=False,
        replace_with_url="",
        replace_with_email="",
        replace_with_phone_number="",
        replace_with_number="",
        replace_with_digit="0",
        replace_with_currency_symbol="",
    ))

    text = hazm.Normalizer().normalize(cleantext)
    text = wierd_pattern.sub(r'', text)
    text = re.sub("#", "", text)
    text = re.sub("\s+", " ", text)

    return text

In [14]:
documents = SimpleDirectoryReader(input_files=["/content/data/input 1.docx"]).load_data()
preprocessed_docs = [cleaning(docs.text) for docs in documents]
preprocessed_docs

فصل دوم. ضوابط و مقررات عام محدوده‌ی شهر تهران







(5-2 در ﮐﻠﯿﻪ زﯾﺮﭘﻬﻨﻪ ﻫﺎي ﺳـﮑﻮﻧﺖ ﻋﺎم)۱(R ﺑﻪ ﻣﻨﻈﻮر ﻣﻤﺎﻧﻌﺖ از ﺗﻔﮑﯿﮏ اراﺿــﯽ و اﻣﻼك در ﻗﻄﻌﺎﺗﯽ ﺑﺎ وﺳ ـﻌﺖ ﺳ ـﻪ ﺑﺮاﺑﺮ ﺣﺪاﻗﻞ اﻧﺪازه ﻗﻄﻌﻪ، ﯾﮏ ﻃﺒﻘﻪ ﺗﺸـﻮﯾﻘﯽ ﺑﺎ ﺗﺮاﮐﻢ ﻣﺘﻨﺎﻇﺮ و در ﻗﻄﻌﺎﺗﯽ ﺑﺎ وﺳـﻌﺖ ﭘﻨﺞ ﺑﺮاﺑﺮ ﺣﺪاﻗﻞ اﻧﺪازه ﻗﻄﻌﻪ، دو ﻃﺒﻘﻪ ﺗﺸـﻮﯾﻘﯽ ﺑﺎ ﺗﺮاﮐﻢ ﻣﺘﻨﺎﻇﺮ ﻣﺠﺎز اﺳـﺖ. ﺑﺮاي ﻣﻮاردي ﮐﻪ ﺗﻌﺪاد ﻃﺒﻘﺎت، 12 ﻃﺒﻘﻪ و ﺑﯿﺸـﺘﺮ ﺑﺎﺷـﺪ، ﻣﻨﻮط ﺑﻪ ﺗﻬﯿﻪ ﻃﺮح ﺗﻮﺟﯿﻬﯽ و ﺗﺼـﻮﯾﺐ در ﮐﻤﯿﺴﯿﻮن ﻣﺎده ﭘﻨﺞ، اﺳﺖ

(6-3 در ﮐﻠﯿﻪ زﯾﺮﭘﻬﻨﻪﻫﺎي ﻓﻌﺎﻟﯿﺖ )۱(S و )۲(S، ﺑﻪ ﻣﻨﻈﻮر ﻣﻤﺎﻧﻌﺖ از ﺗﻔﮑﯿﮏ اراﺿـﯽ و اﻣﻼك، در ﻗﻄﻌﺎﺗﯽ ﺑﺎ وﺳـﻌﺖ ﺳـﻪ ﺑﺮاﺑﺮ ﺣﺪاﻗﻞ اﻧﺪازه ﻗﻄﻌﻪ، ﯾﮏ ﻃﺒﻘﻪ ﺗﺸـﻮﯾﻘﯽ ﺑﺎ ﺗﺮاﮐﻢ ﻣﺘﻨﺎﻇﺮ و در ﻗﻄﻌﺎﺗﯽ ﺑﺎ وﺳـﻌﺖ ﭘﻨﺞ ﺑﺮاﺑﺮ ﺣﺪاﻗﻞ اﻧﺪازه ﻗﻄﻌﻪ، دو ﻃﺒﻘﻪ ﺗﺸـﻮﯾﻘﯽ ﺑﺎ ﺗﺮاﮐﻢ ﻣﺘﻨﺎﻇﺮ ﻣﺠﺎز اﺳـﺖ. ﺑﺮاي ﻣﻮاردي ﮐﻪ ﺗﻌﺪاد ﻃﺒﻘﺎت، 12 ﻃﺒﻘﻪ و ﺑﯿﺸـﺘﺮ ﺑﺎﺷـﺪ، ﻣﻨﻮط ﺑﻪ ﺗﻬﯿﻪ ﻃﺮح ﺗﻮﺟﯿﻬﯽ و ﺗﺼـﻮﯾﺐ درﮐﻤﯿﺴﯿﻮن ﻣﺎده ﭘﻨﺞ، اﺳﺖ

(11-4 در ﮐﻠﯿﻪ زﯾﺮﭘﻬﻨﻪﻫﺎي ﻣﺨﺘﻠﻂ )۱(M، ﺑﻪ ﻣﻨﻈﻮر ﻣﻤﺎﻧﻌﺖ از ﺗﻔﮑﯿﮏ اراﺿـﯽ و اﻣﻼك، در ﻗﻄﻌﺎﺗﯽ ﺑﺎ وﺳـﻌﺖ ﺳـﻪ ﺑﺮاﺑﺮ ﺣﺪاﻗﻞ اﻧﺪازه ﻗﻄﻌﻪ، ﯾﮏ ﻃﺒﻘﻪ ﺗﺸـﻮﯾﻘﯽ ﺑﺎ ﺗﺮاﮐﻢ ﻣﺘﻨﺎﻇﺮ و در ﻗﻄﻌﺎﺗﯽ ﺑﺎ وﺳـﻌﺖ ﭘﻨﺞ ﺑﺮاﺑﺮ ﺣﺪاﻗﻞ اﻧﺪازه ﻗﻄﻌﻪ، دو ﻃﺒﻘ

In [17]:
query_documents = SimpleDirectoryReader(input_files=["/content/data/questions 1.docx"]).load_data()
preprocessed_questions = [cleaning(query.text) for query in query_documents]
preprocessed_questions

پاسخ

سوال

با سلام و احترام؛ لطفا به شهرداری منطقه مراجعه فرمائید. 



باسلام اینجانب دارای 300 زمین مشاع در پلاک ثبتی منطقه 5 شهرداری تهران ناحیه 3 مراداباد می باشم درخواست جواز ساخت را از ان مقامات مجترم تقاضامندم با توجه به اینکه پلاک فوق بمرور زمان در داخل شهر تهران قرارگرفته لذا تقاضامندم بزرگواران در این مورد دستورات لازم را صادر وامر به ابلاغ فرمایند با تقدیم احترامات فایقه رحیم جوادی



با سلام و احترام؛ صدور پروانه صرفا درچهارچوب ضوابط طرح تفصیلی امکان پذیر می باشد. 



با سلام احترام خدمت شما من سال پیش در منطقه ۱۴ در ریزدانه ملکی خریداری کردم به متراژ ۷۲متر چون تشکیل پرونده من برای تخریب و نوسازی بعد از برج ۵ بود طبقه چهارم و ۲+به این پرونده تعلیق نگرفته میخواستم بدونم صبر کنم شهرداری و شورای شهر تصمیم به برگردوندن به روال قبل رو داره آیا امیدی هست یا بازم گرفتاریم شماره پرونده ۱۴۰۱۰۲۷۵۶ ممنون از پاسخ شما



با سلام و احترام؛  در نسخه دیجیتالی اخذ پرینت امکان پذیر می باشد. 



چگونه می توانم از صورتجلسات کمیسیون ماه 5 پرینت بگیرم



با سلام و احترام تا این تاریخ شیوه نامه م

In [27]:
Settings.embed_model = embed_model
Settings.llm = llm
index = VectorStoreIndex.from_documents(
    documents,
)
query_engine = index.as_query_engine(similarity_top_k=3)
response = query_engine.query( preprocessed_questions[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [28]:
print(response)

The second chapter outlines the specifications and limitations for buildings in Tehran. Here are some details:

- Buildings with a height of 6 floors: 1 (r) is implemented to prevent the separation of units and properties.

- Buildings with a height of 12 floors: 1 (r) and 1 (M) are implemented to prevent the separation of units and properties.

- Buildings with a height of 20 floors: 2 sections with a height of 12 floors and 2 sections with a height of 8 floors are allowed.

- Buildings with a height of 30 floors: 2 sections with a height of 20 floors are allowed.

Additionally, in the whole complex:

- (5-2) for preventing the separation of units and properties, in the sections with a height of 6 floors, and in the sections with a height of 12 floors, 1 (r) is implemented.

- (11-4) for preventing the separation of units and properties, in the sections with a height of 6 floors, and in the sections with a height of 12 floors, 
